In [207]:
import datetime
import requests
import pandas as pd

In [208]:
companies = ['RELIANCE', 'BAJAJFINSV', 'HDFCBANK', 'HDFC', 'ITC', 'INFY', 'ADANIPORTS', 'ICICIBANK', 'BAJFINANCE', 'TCS', 'KOTAKBANK', 'LT', 'AXISBANK', 'MARUTI', 'SBIN', 'TATAMOTERS', 'TATASTEEL', 'HINDALCO', 'BHARTIARTL', 'NTPC', 'EICHERMOT', 'APOLLOHOSP', 'M&N', 'INDUSINDBK', 'ASIANPAINT', 'HINDUNILVR', 'ONGC', 'BAJAJ-AUTO', 'HEROMOTOCO', 'TITAN', 'WIPRO', 'HDFCLIFE', 'SHREECEM', 'ULTRACEMCO', 'TECHM', 'SUNPHARMA', 'CIPLA', 'HCLTECH', 'TATACONSUM', 'BPCL', 'UPL', 'SBILIFE', 'JSWSTEEL', 'POWERGRID', 'DIVISLAB', 'COALINDIA', 'GRASIM', 'DRREDDY', 'BRITANNIA', 'MESTLEIND']

In [209]:
# set up today and yesterday
current_date = datetime.date.today()
today = current_date.strftime("%d%m%Y")


yesterday = current_date - datetime.timedelta(days=1)
yesterday = yesterday.strftime('%d%m%Y')


In [210]:
# setup the source url to fetch the new data from nseindia.com
source_file_name = 'sec_bhavdata_full_{}.csv'
base_url = r'https://archives.nseindia.com/products/content/'
source_url = base_url+source_file_name.format(yesterday)
source_url

'https://archives.nseindia.com/products/content/sec_bhavdata_full_13102022.csv'

In [211]:
# setup the column names of the new file
column_names = ['Symbol', 'Series', 'Date', 'PrevClose', 'OpenPrice', 'HighPrice', 'LowPrice', 'LastPrice', 'ClosePrice'+yesterday, 'AvgPrice', 'TotalTradedQuantity'+yesterday, 'TurnoverInLacs'+yesterday, 'NoOfTrades', 'DeliveryQuantity'+yesterday, 'DeliveryPercentage'+yesterday]

In [212]:
main_bhav_data = pd.read_csv(source_url, names=column_names)

In [213]:
main_bhav_data = main_bhav_data.drop(columns=['Date','PrevClose', 'OpenPrice', 'HighPrice', 'LowPrice', 'LastPrice', 'AvgPrice', 'NoOfTrades'])

In [214]:
main_bhav_data = main_bhav_data.loc[main_bhav_data['Symbol'].isin(companies)]

In [215]:
main_bhav_data

,Symbol,Series,ClosePrice13102022,TotalTradedQuantity13102022,TurnoverInLacs13102022,DeliveryQuantity13102022,DeliveryPercentage13102022
55,ADANIPORTS,EQ,784.75,7491563,59083.82,1510646,20.16
137,APOLLOHOSP,EQ,4266.05,277038,11822.66,134839,48.67
171,ASIANPAINT,EQ,3209.00,779890,25075.25,356092,45.66
203,AXISBANK,EQ,802.00,13024911,104649.02,7174692,55.08
217,BAJAJ-AUTO,EQ,3604.80,336605,12176.42,175361,52.10
220,BAJAJFINSV,EQ,1681.75,1065166,17876.84,248944,23.37
224,BAJFINANCE,EQ,7151.65,608655,43572.21,157860,25.94
281,BHARTIARTL,EQ,768.90,2842056,21903.33,1519596,53.47
305,BPCL,EQ,300.30,3029077,9128.44,1334588,44.06
309,BRITANNIA,EQ,3784.60,156770,5919.31,94118,60.04


In [216]:
company_count = {}
for company in companies:
    company_count[company] = 0


In [217]:
is_valid = [0 for _ in main_bhav_data.index]

In [218]:
main_bhav_data.insert(1, 'is_valid', is_valid)

In [219]:
for ind in main_bhav_data.index:
    company = main_bhav_data.at[ind, 'Symbol']
    if not company_count[company]:
        company_count[company] = 1
        main_bhav_data.at[ind, 'is_valid'] = 1

In [220]:
main_bhav_data = main_bhav_data.loc[main_bhav_data['is_valid'] == 1]

In [221]:
main_bhav_data.to_csv("nse_data_file.csv", index=False)

In [222]:
main_bhav_data = pd.read_csv('nse_data_file.csv')

In [223]:
source_url = base_url+source_file_name.format(today)

In [224]:
# setup the column names of the new file
column_names = ['Symbol', 'Series', 'Date', 'PrevClose', 'OpenPrice', 'HighPrice', 'LowPrice', 'LastPrice', 'ClosePrice'+today, 'AvgPrice', 'TotalTradedQuantity'+today, 'TurnoverInLacs'+today, 'NoOfTrades', 'DeliveryQuantity'+today, 'DeliveryPercentage'+today]

In [225]:
# fetch today's bhav data file from nseindia.com
today_bhav_data = pd.read_csv(source_url, names=column_names)

In [226]:
today_bhav_data = today_bhav_data.drop(columns=['Date','PrevClose', 'OpenPrice', 'HighPrice', 'LowPrice', 'LastPrice', 'AvgPrice', 'NoOfTrades'])

In [227]:
today_bhav_data = today_bhav_data.loc[today_bhav_data['Symbol'].isin(companies)]

In [228]:
company_count = {}
for company in companies:
    company_count[company] = 0

In [229]:
is_valid = [0 for _ in today_bhav_data.index]

In [230]:
today_bhav_data.insert(1, 'is_valid', is_valid)

In [231]:
for ind in today_bhav_data.index:
    company = today_bhav_data.at[ind, 'Symbol']
    if not company_count[company]:
        company_count[company] = 1
        today_bhav_data.at[ind, 'is_valid'] = 1

In [232]:
today_bhav_data = today_bhav_data.loc[today_bhav_data['is_valid'] == 1]

In [233]:
today_bhav_data.to_csv('temp_nse_data.csv', index=False)

In [234]:
today_bhav_data = pd.read_csv('temp_nse_data.csv')

In [235]:
today_bhav_data

,Symbol,is_valid,Series,ClosePrice14102022,TotalTradedQuantity14102022,TurnoverInLacs14102022,DeliveryQuantity14102022,DeliveryPercentage14102022
0,ADANIPORTS,1,EQ,780.10,4562526,36025.51,650375,14.25
1,APOLLOHOSP,1,EQ,4297.15,254366,10930.27,94375,37.10
2,ASIANPAINT,1,EQ,3185.50,845765,27163.17,511708,60.50
3,AXISBANK,1,EQ,800.50,8220645,66349.42,3810137,46.35
4,BAJAJ-AUTO,1,EQ,3570.50,499983,18069.55,256918,51.39
5,BAJAJFINSV,1,EQ,1688.30,1239168,21010.99,387881,31.30
6,BAJFINANCE,1,EQ,7272.50,839012,61029.68,267321,31.86
7,BHARTIARTL,1,EQ,765.95,3608210,27861.01,2176149,60.31
8,BPCL,1,EQ,297.85,2577730,7747.98,1166818,45.27
9,BRITANNIA,1,EQ,3796.25,350141,13294.52,253321,72.35


In [236]:
# setup the column names for yesterday's data from the main bhav data file.
header_main = ['ClosePrice'+yesterday, 'TotalTradedQuantity'+yesterday, 'TurnoverInLacs'+yesterday, 'DeliveryQuantity'+yesterday, 'DeliveryPercentage'+yesterday]
header_main

['ClosePrice13102022',
 'TotalTradedQuantity13102022',
 'TurnoverInLacs13102022',
 'DeliveryQuantity13102022',
 'DeliveryPercentage13102022']

In [237]:
# setup the column names for today's data from the nseindia.com
header_new = ['ClosePrice'+today, 'TotalTradedQuantity'+today, 'TurnoverInLacs'+today, 'DeliveryQuantity'+today, 'DeliveryPercentage'+today]
header_new

['ClosePrice14102022',
 'TotalTradedQuantity14102022',
 'TurnoverInLacs14102022',
 'DeliveryQuantity14102022',
 'DeliveryPercentage14102022']

In [238]:
# extract the new data and paste to the main bhav data file.
for i in range(5):
    new_head = header_new[i]
    prev_head = main_bhav_data.columns.get_loc(header_main[i]) + 1

    extracted_col = today_bhav_data[new_head]
    main_bhav_data.insert(prev_head, new_head, extracted_col)
    

In [239]:
main_bhav_data

,Symbol,is_valid,Series,ClosePrice13102022,ClosePrice14102022,TotalTradedQuantity13102022,TotalTradedQuantity14102022,TurnoverInLacs13102022,TurnoverInLacs14102022,DeliveryQuantity13102022,DeliveryQuantity14102022,DeliveryPercentage13102022,DeliveryPercentage14102022
0,ADANIPORTS,1,EQ,784.75,780.10,7491563,4562526,59083.82,36025.51,1510646,650375,20.16,14.25
1,APOLLOHOSP,1,EQ,4266.05,4297.15,277038,254366,11822.66,10930.27,134839,94375,48.67,37.10
2,ASIANPAINT,1,EQ,3209.00,3185.50,779890,845765,25075.25,27163.17,356092,511708,45.66,60.50
3,AXISBANK,1,EQ,802.00,800.50,13024911,8220645,104649.02,66349.42,7174692,3810137,55.08,46.35
4,BAJAJ-AUTO,1,EQ,3604.80,3570.50,336605,499983,12176.42,18069.55,175361,256918,52.10,51.39
5,BAJAJFINSV,1,EQ,1681.75,1688.30,1065166,1239168,17876.84,21010.99,248944,387881,23.37,31.30
6,BAJFINANCE,1,EQ,7151.65,7272.50,608655,839012,43572.21,61029.68,157860,267321,25.94,31.86
7,BHARTIARTL,1,EQ,768.90,765.95,2842056,3608210,21903.33,27861.01,1519596,2176149,53.47,60.31
8,BPCL,1,EQ,300.30,297.85,3029077,2577730,9128.44,7747.98,1334588,1166818,44.06,45.27
9,BRITANNIA,1,EQ,3784.60,3796.25,156770,350141,5919.31,13294.52,94118,253321,60.04,72.35
